In [11]:
import pandas as pd

In [12]:
import gensim

In [13]:
df = pd.read_csv('/content/train.txt',delimiter=';',names=['sentence','label'])

In [14]:
dff = pd.read_csv('/content/test.txt',delimiter=';',names=['sentence','label'])

In [15]:
df.shape

(16000, 2)

In [16]:
dff.shape

(2000, 2)

In [17]:
dff.head()

,sentence,label
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [18]:
dft=df.append(dff)

In [19]:
df.head()

,sentence,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [20]:
dft.shape

(18000, 2)

In [21]:
from nltk.corpus import stopwords

In [22]:
import nltk

In [23]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [25]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [26]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)
            
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))
    
            
    return " ".join(y)

In [27]:
dft['transformed_text']=dft['sentence'].apply(transform_text)
    

In [28]:
dft['label'].value_counts()

joy         6057
sadness     5247
anger       2434
fear        2161
love        1463
surprise     638
Name: label, dtype: int64

In [29]:
dft.head()

,sentence,label,transformed_text
0,i didnt feel humiliated,sadness,didnt feel humili
1,i can go from feeling so hopeless to so damned...,sadness,go feel hopeless damn hope around someon care ...
2,im grabbing a minute to post i feel greedy wrong,anger,im grab minut post feel greedi wrong
3,i am ever feeling nostalgic about the fireplac...,love,ever feel nostalg fireplac know still properti
4,i am feeling grouchy,anger,feel grouchi


In [30]:
data = dft[['transformed_text','label']]

In [31]:
data.head()

,transformed_text,label
0,didnt feel humili,sadness
1,go feel hopeless damn hope around someon care ...,sadness
2,im grab minut post feel greedi wrong,anger
3,ever feel nostalg fireplac know still properti,love
4,feel grouchi,anger


In [32]:
X = data.iloc[:12000,0:1]
y = data['label']
y = y.iloc[:12000]

In [33]:
X

,transformed_text
0,didnt feel humili
1,go feel hopeless damn hope around someon care ...
2,im grab minut post feel greedi wrong
3,ever feel nostalg fireplac know still properti
4,feel grouchi
...,...
11995,realli good first impress feel one dont realli...
11996,im feel artist googl art project
11997,feel innoc world go becom great friend
11998,feel though ive rob much summer pleasant altho...


In [34]:
y

0        sadness
1        sadness
2          anger
3           love
4          anger
          ...   
11995        joy
11996        joy
11997        joy
11998        joy
11999        joy
Name: label, Length: 12000, dtype: object

In [35]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

y = encoder.fit_transform(y)

In [36]:
y

array([4, 4, 0, ..., 2, 2, 2])

In [37]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

# BOW

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
cv = CountVectorizer()

In [40]:
X_train_bow = cv.fit_transform(X_train['transformed_text']).toarray()
X_test_bow = cv.transform(X_test['transformed_text']).toarray()

In [41]:
X_train_bow.shape

(9600, 8063)

In [42]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [43]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.335

In [44]:
confusion_matrix(y_test,y_pred)

array([[ 92,  81,  38,  37,  45,  25],
       [ 19, 124,  30,  35,  39,  37],
       [ 79, 101, 237, 151, 130,  75],
       [ 14,  26,  38,  87,  30,  26],
       [121, 143,  57,  89, 240,  59],
       [  7,  16,  20,  11,  17,  24]])

In [45]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.835

In [46]:
cv = CountVectorizer(max_features=3000)

X_train_bow = cv.fit_transform(X_train['transformed_text']).toarray()
X_test_bow = cv.transform(X_test['transformed_text']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8345833333333333

In [47]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['transformed_text']).toarray()
X_test_bow = cv.transform(X_test['transformed_text']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8479166666666667

# TFIDF

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
tfidf = TfidfVectorizer()

In [50]:
X_train_tfidf = tfidf.fit_transform(X_train['transformed_text']).toarray()
X_test_tfidf = tfidf.transform(X_test['transformed_text'])

In [51]:
rf = RandomForestClassifier()

rf.fit(X_train_tfidf,y_train)
y_pred = rf.predict(X_test_tfidf)

accuracy_score(y_test,y_pred)

KeyboardInterrupt: ignored

# Word2vec

In [55]:
import gensim
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [56]:
sent = [row.split() for row in X['transformed_text']]

In [57]:
len(sent)

12000

In [ ]:
# from gensim.models.phrases import Phrases, Phraser

In [ ]:
# phrases = Phrases(sent, min_count=30, progress_per=500)

In [ ]:
# bigram = Phraser(phrases)

In [ ]:
# sentences = bigram[sent]

In [ ]:
# sentences

In [58]:
model = gensim.models.Word2Vec(
    sent,
    window=5,
    min_count=2
)

In [59]:
model

In [ ]:
#  model.build_vocab(sent)

In [ ]:
# model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)

In [ ]:
model.corpus_count

In [60]:
model.wv.similar_by_word('man')

[('watch', 0.9999480247497559),
 ('sens', 0.9999419450759888),
 ('part', 0.9999303221702576),
 ('may', 0.9999300241470337),
 ('hand', 0.999929666519165),
 ('see', 0.9999293684959412),
 ('show', 0.9999293088912964),
 ('find', 0.9999291896820068),
 ('turn', 0.9999285936355591),
 ('well', 0.9999284148216248)]

In [64]:
# model.wv.index_to_key

AttributeError: ignored

In [61]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [62]:
from tqdm import tqdm
import numpy as np

In [63]:
c = []
for doc in tqdm(X['transformed_text'].values):
    c.append(document_vector(doc))

  0%|          | 0/12000 [00:00<?, ?it/s]


AttributeError: ignored

In [ ]:
C= np.array(c)

In [ ]:
C.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(C,y,test_size=0.2,random_state=1)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)